# NYC Traffic Crashes Analysis and Data Preparation

This notebook performs comprehensive analysis of NYC traffic crashes data, preparing it for visualization in a web application. The analysis includes data cleaning, integration, and exploratory analysis of crash patterns across New York City.

# 1. Data Loading and Initial Inspection

First, we'll import the required libraries and load our datasets.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

# Set display options
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
plt.style.use('seaborn')